# <br>




In [1]:
spark.version

3.1.2

In [2]:
val data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

In [9]:
SELECT * from delta.`/tmp/delta-table`

java.lang.LinkageError: loader constraint violation: when resolving method 'java.lang.Object org.apache.spark.sql.catalyst.parser.ParserUtils$.withOrigin(org.antlr.v4.runtime.ParserRuleContext, scala.Function0)' the class loader polynote.kernel.util.LimitedSharingClassLoader @5e835200 of the current class, io/delta/sql/parser/DeltaSqlAstBuilder, and the class loader 'app' for the method's defining class, org/apache/spark/sql/catalyst/parser/ParserUtils$, have different Class objects for the type org/antlr/v4/runtime/ParserRuleContext used in the signature (io.delta.sql.parser.DeltaSqlAstBuilder is in unnamed module of loader polynote.kernel.util.LimitedSharingClassLoader @5e835200, parent loader 'app'; org.apache.spark.sql.catalyst.parser.ParserUtils$ is in unnamed module of loader 'app')